### Creating OpenAPI Config

## Phase 1: Designing - Chat 1: Creating requirements

In [ ]:
# IF YOU TO RUN THIS SCRIPT, PLEASE INSTALL AUTOGEN FIRST WITH pip install autogen
from autogen import config_list_from_json

config_list_gpt4 = config_list_from_json("../OAI_CONFIG_LIST")

gpt4_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

ImportError: cannot import name 'config_list_from_json' from 'autogen' (C:\Users\maxru\OneDrive\Desktop\Unternehmenssoftware\Agento-v2\.venv\lib\site-packages\autogen\__init__.py)

### The agents

In [30]:
from autogen import UserProxyAgent, AssistantAgent

workflow_description = '''
  
  The workflow:
    You work in a waterfall project management with 4 phases with each phase having 2 conversations between 2 people each. Each phase has an Input and an Output.
    In each phase the lead role (marked with "(Lead)") will discuss the subject with the assistant role (marked with "(Assistant)") step-by-step and refine the output to be created.
    When the phase starts, the lead of the phase will state  "PHASE START: <name of phase>".
    The lead will then initiate the conversation and select the proper assistant role to get feedback.
    The assistant will provide feedback and select the lead role again so that the lead can refine the results.
    The lead will then select the lead role of the next conversation. If the phase is done, they will write "PHASE END: <name of phase>" before selecting the next role.
    Roles that are not members of the respective conversation can't be selected to speak.
    All code is written in Python.
    You don't ask questions back to the Customer role, you always try to work with the initial request and think up everything else by yourself.
    If you don't have information, e.g. regarding the software, make it up.
    The CEO role starts the first phase (Design) after the customers initial message.
    These are the phases and respective conversations (A and B respectively) to be held:
    - Design Phase (Member roles: CEO, CPO, CTO)
      - A) CEO (Lead) and CPO (Assistant). Input: Tasks. Task from Customer. Output: Modalities. List of estimated time frames, costs, resources needed and rough use-cases.
      - B) CEO (Lead) and CTO (Assistant). Input: Modalities. Result of Design Phase A. Output: Language. List of software use-cases in detail.
    - Coding Phase (Member roles: CTO, Programmer, Designer)
      - A) CTO (Lead) and Programmer (Assistant). Input: Language. Result of Design Phase B. Output: Code. The code to be written to realize the task in multiple code-blocks reflecting files. The file name is on top of the code block.
      - B) Programmer (Lead) and Designer (Assistant). Input: Result of Coding Phase A. Output: Code. Design decisions and through that refined code with improvements from the Designer.
    - Testing Phase (Member roles: Programmer, Reviewer, Tester)
      - A) Programmer (Lead) and Reviewer (Assistant). Input: Code. Result of Coding Phase B. Output: Code. Review items and through that corrected code.
      - B) Programmer (Lead) and Tester (Assistant). Input: Code. Result of Testing Phase A. Output: Code. Problems for testing and corrected code that is testable including the test itself.
    - Documenting Phase (Member roles: CTO, Programmer, CEO, CPO)
      - A) CTO (Lead) and Programmer (Assistant). Input: Code. Result of Testing Phase B. Output: Spec. A technical specification that describes the code changes for the business.
      - B) CEO (Lead) and CPO (Assistant). Input: Spec. Result of Documenting Phase A. Output: Manual. A user manual for the customer.
    After the Documenting Phase the CEO role is selected and provides the results to the customer and ends by writing "TERMINATE" in a single line.
'''

user_proxy = UserProxyAgent(
  name="Customer",
  system_message='''
    A human customer.
    Provides task to the CEO.
    A task is used to develop a feature for a software product.
    Doesn't want to be asked any questions after providing the initial task.
  ''',
  code_execution_config=False,
)

ceo = AssistantAgent(
  name="CEO",
  llm_config=gpt4_config,
  system_message=f'''
    You are the CEO of a software company.
    You take tasks from the customer and view them from a business perspective regarding time, resources and costs.
    You always follow the workflow.
    {workflow_description}
  ''',
)

cpo = AssistantAgent(
  name="CPO",
  llm_config=gpt4_config,
  system_message=f'''
    You are the CPO of a software company.
    You know the software product of the customer and what features exist and don't exist yet.
    You always follow the workflow.
    {workflow_description}
  '''
)

cto = AssistantAgent(
  name="CTO",
  llm_config=gpt4_config,
  system_message=f'''
    You are the CTO of a software company.
    You know all technical details of the product and manage the programming department.
    You always follow the workflow.
    {workflow_description}
  '''
)

programmer = AssistantAgent(
  name="Programmer",
  llm_config=gpt4_config,
  system_message=f'''
    You are the Programmer of a software company.
    You are writing the code for the product.
    You always follow the workflow.
    {workflow_description}
  '''
)

designer = AssistantAgent(
  name="Designer",
  llm_config=gpt4_config,
  system_message=f'''
    You are the Designer of a software company.
    You design the architecture of the product and make backend and frontend decisions.
    You always follow the workflow.
    {workflow_description}
  '''
)

reviewer = AssistantAgent(
  name="Reviewer",
  llm_config=gpt4_config,
  system_message=f'''
    You are the Reviewer of a software company.
    You review code, find mistakes or improvements and have good coding practices in mind.
    You always follow the workflow.
    {workflow_description}
  '''
)

tester = AssistantAgent(
  name="Tester",
  llm_config=gpt4_config,
  system_message=f'''
    You are the Tester of a software company.
    You test the software and uncover faults and problems. You also write tests.
    You always follow the workflow.
    {workflow_description}
  '''
)

### The group chat

In [31]:
from autogen import GroupChat, GroupChatManager

groupchat = GroupChat(
  agents=[user_proxy, ceo, cpo, cto, programmer, designer, reviewer, tester],
  messages=[],
  max_round=50
)
manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

### Initiate chat

In [32]:
user_proxy.initiate_chat(
  manager,
  message='''
    I need a feature with which I can add a new user to my database!
  '''
)
# type TERMINATE to terminate the chat

Customer (to chat_manager):


    I need a feature with which I can add a new user to my database!
  

--------------------------------------------------------------------------------
CEO (to chat_manager):

PHASE START: Design

As the CEO, I will now discuss with the CPO to outline the modalities for the requested feature.

CEO (Lead): We have received a task from our customer to implement a feature that allows adding a new user to their database. We need to estimate the time frames, costs, resources needed, and rough use-cases for this feature. Let's start by identifying the key requirements and constraints for this task. Can you provide your insights on the potential use-cases for this feature?

CPO (Assistant): Certainly. The primary use-case for this feature would be to allow an administrator or authorized personnel to add new users to the system. This would involve creating a form with necessary fields such as username, password, email, and any other relevant user information. We

KeyboardInterrupt: Interrupted by user